In [1]:
import numpy as np
import tensorflow as tf
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from CocktailEmbeddingMaker import CocktailEmbeddingMaker
from CocktailEmbeddingMaker import Eval
import wandb
# 데이터 로드
with open('./train_data.json', 'r') as f:
    json_data = json.load(f)
with open('../flavor.json', 'r') as f:
    flavor_data = json.load(f)
test_user_list = []

attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky']
user_num = 5
for i in range(user_num):
    user = {}
    user['user_id'] = i
    preference = {}
    user['ABV'] = np.random.randint(0,60)
    for attribute in attributes[2:]:
        user[attribute] = np.random.randint(0,100)
    test_user_list.append(user)
class RecipeGenerationModel:
    #RecipeGenerationModel(cocktail_embedding_maker, wandb_flag=True, max_recipe_length=10)
    def __init__(self, cocktail_embedding_maker,wandb_Flag=False, max_recipe_length=10):
        self.cocktail_embedding_maker = cocktail_embedding_maker
        self.ingredient_ids = cocktail_embedding_maker.ingredient_ids
        self.num_ingredients = cocktail_embedding_maker.num_ingredients
        self.max_recipe_length = max_recipe_length
        self.ingredient_embedding_matrix = cocktail_embedding_maker.create_ingredient_embedding_matrix()
        self.sweep_config = None
        self.evaluation_metrics=None
        self.sweep_id = None
        self.wandb = wandb_Flag
        self.model = self.build_model()
        self.total_amount = 200
        self.Eval = Eval(json_data,flavor_data,self.total_amount)
        self.attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky']
        self.evaluation_metrics = ['diversity', 'abv_match', 'taste_match']


    def build_model(self):
        model = Sequential([
            Embedding(self.num_ingredients, self.ingredient_embedding_matrix.shape[1],
                      weights=[self.ingredient_embedding_matrix], input_length=self.max_recipe_length, trainable=False),
            LSTM(128, return_sequences=True),
            LSTM(128),
            Dense(64, activation='gelu'),
            Dense(self.num_ingredients, activation='softmax')
        ])
        model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
        return model

    def train(self, recipes, epochs=50, batch_size=32,learning_rate=0.001):
        ingredient_sequences = []
        next_ingredients = []

        for recipe in recipes:
            sequence = [self.ingredient_ids[self.cocktail_embedding_maker.normalize_string(ingredient)] for ingredient in recipe]
            for i in range(1, len(sequence)):
                ingredient_sequences.append(sequence[:i])
                next_ingredients.append(sequence[i])

        ingredient_sequences = tf.keras.preprocessing.sequence.pad_sequences(ingredient_sequences, maxlen=self.max_recipe_length)
        next_ingredients = tf.keras.utils.to_categorical(next_ingredients, num_classes=self.num_ingredients)
        evaluation_interval = 5
        if self.wandb:
            # wandb 초기화
            wandb.init(project='cocktail_recipe_generation')


        # for epoch in range(epochs):
        history = self.model.fit(ingredient_sequences, next_ingredients, epochs=epochs, batch_size=batch_size, verbose=0)
        loss = history.history['loss'][0]
        accuracy = history.history['accuracy'][0]
        if self.wandb:
            # wandb 로깅
            wandb.log({
                'epoch': epochs,
                'loss': loss,
                'accuracy': accuracy,
            })
        # eval을 호출 해서 평가를 수행하고 결과를 evaluation_result에 저장한다. 
        # 저장후 evaluation_metrics에 지정된 평가 지표 합을 계산해서 performance변수에 저장한다. 
        #wandb에 performance와 개별 평가 지표결과를 로깅한다. 
            # 모델 평가
        print("evaluating model")

        evaluation_results = self.Eval.evaluate_model(self.model, test_user_list)
        
        # 평가 지표 계산
        performance = sum(evaluation_results[metric] for metric in self.evaluation_metrics)
        

        if self.wandb:
            # 평가 지표 로깅
            wandb.log({'performance': performance, **evaluation_results})
            wandb.finish()
        return loss, accuracy, performance





# # 모델 학습
# recipe_generation_model.train(train_recipes, epochs=50, batch_size=32)
def train_with_sweep():
    # CocktailEmbeddingMaker 인스턴스 생성
    cocktail_embedding_maker = CocktailEmbeddingMaker(json_data, flavor_data)

    # RecipeGenerationModel 인스턴스 생성
    recipe_generation_model = RecipeGenerationModel(cocktail_embedding_maker, True, max_recipe_length=10)

    # 학습 데이터 준비
    train_recipes = [recipe['recipe'].keys() for recipe in json_data['cocktail_info']]

    # 모델 학습
    loss, accuracy, performance = recipe_generation_model.train(train_recipes)
    return loss, accuracy, performance



sweep_configuration = {
    'method': 'random',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'performance'},
    'parameters': 
    {
        'batch_size': {'values': [16, 32, 64]},
        'epochs': {'values': [30, 40, 50]},
        'lr': {'max': 0.1, 'min': 0.01}
     }
}


sweep_id = wandb.sweep(sweep_configuration, project='cocktail_recipe_generation')
wandb.agent(sweep_id, function=train_with_sweep)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: a144h0ce
Sweep URL: https://wandb.ai/jennyshin_gist_2024/cocktail_recipe_generation/sweeps/a144h0ce


wandb: Agent Starting Run: e5yv8gp5 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 15
wandb: 	lr: 0.06412994046039155
c:\Users\aimed-lab-1\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ann4622-sin (jennyshin_gist_2024). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


evaluating model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms

abv_match,▁
accuracy,▁
diversity,▁
epoch,▁
loss,▁
performance,▁
similarity,▁
taste_match,▁
abv_match,0.84598
accuracy,0.03032
diversity,0.80289


...

abv_match,▁
accuracy,▁
diversity,▁
epoch,▁
loss,▁
performance,▁
similarity,▁
taste_match,▁
abv_match,0.70814
accuracy,0.03664
diversity,0.80288


wandb: Agent Starting Run: 2wkov3a6 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.03634762218534171
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


evaluating model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms

abv_match,▁
accuracy,▁
diversity,▁
epoch,▁
loss,▁
performance,▁
similarity,▁
taste_match,▁
abv_match,0.54186
accuracy,0.0259
diversity,0.80288


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:



# 새로운 레시피 생성
import random
total_amount = 200
seed_ingredient=random.choice(list(cocktail_embedding_maker.ingredient_ids.keys()))
user_preference = test_user_list[0]
generated_recipe = recipe_generation_model.generate_recipe(seed_ingredient, user_preference, max_length=5)
generated_amount = [ratio * total_amount for ratio in generated_recipe[1]]
result = []
recipe = {}
for item,quantity_ratio in zip(generated_recipe[0],generated_recipe[1]):
    recipe[item] = quantity_ratio * total_amount
    result.append(f"{item} ={quantity_ratio * total_amount}")
print(f"Generated ingredient list: {generated_recipe[0]}")
print(f"Generated Recipe : {result}")
test_abv= cocktail_embedding_maker.calculate_recipe_abv(generated_recipe[0], generated_recipe[1])
taste_info_result = cocktail_embedding_maker.get_taste_info(recipe)
taste_info_result.pop('ID')
print(f"taste_info_result: {json.dumps(taste_info_result,indent=4)}")
print(f"user_preference: {json.dumps(user_preference,indent=4)}")
print(f"test_abv: {test_abv}")


# wandb sweep


In [ ]:


# 데이터 로드
with open('./train_data.json', 'r') as f:
    json_data = json.load(f)
with open('../flavor.json', 'r') as f:
    flavor_data = json.load(f)

# CocktailEmbeddingMaker 인스턴스 생성
cocktail_embedding_maker = CocktailEmbeddingMaker(json_data, flavor_data)

# RecipeGenerationModel 인스턴스 생성
recipe_generation_model = RecipeGenerationModel(cocktail_embedding_maker, max_recipe_length=10)

# 학습 데이터 준비
train_recipes = [recipe['recipe'].keys() for recipe in json_data['cocktail_info']]



In [ ]:

# 사용자 선호도 설정
user_preference = {
    'sweet': 80,
    'sour': 61,
    'bitter': 50,
    'fruity': 30,
    'ABV': 30,
    'abv_min': 0,
    'abv_max': 40
}

# 새로운 레시피 생성
import random
total_amount = 200
seed_ingredient=random.choice(list(cocktail_embedding_maker.ingredient_ids.keys()))
generated_recipe = recipe_generation_model.generate_recipe(seed_ingredient, user_preference, max_length=5)
total_amount = 200
generated_amount = [ratio * total_amount for ratio in generated_recipe[1]]
result = []
recipe = {}
for item,quantity_ratio in zip(generated_recipe[0],generated_recipe[1]):
    recipe[item] = quantity_ratio * total_amount
    result.append(f"{item} ={quantity_ratio * total_amount}")
print(f"Generated ingredient list: {generated_recipe[0]}")
print(f"Generated Recipe : {result}")
test_abv= cocktail_embedding_maker.calculate_recipe_abv(generated_recipe[0], generated_recipe[1])
taste_info_result = cocktail_embedding_maker.get_taste_info(recipe)
taste_info_result.pop('ID')
print(f"taste_info_result: {json.dumps(taste_info_result,indent=4)}")
print(f"user_preference: {json.dumps(user_preference,indent=4)}")
print(f"test_abv: {test_abv}")


In [ ]:
# random user preference
test_user_list = []

attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky']
user_num = 1000
for i in range(user_num):
    user = {}
    user['user_id'] = i
    preference = {}
    user['ABV'] = np.random.randint(0,60)
    for attribute in attributes[2:]:
        user[attribute] = np.random.randint(0,100)
    test_user_list.append(user)


In [ ]:
test_user_list[0]


In [ ]:
# evaluate
import random
total_amount = 200
seed_ingredient=random.choice(list(cocktail_embedding_maker.ingredient_ids.keys()))
print(seed_ingredient)
generated_recipe = recipe_generation_model.generate_recipe(seed_ingredient, user_preference, max_length=5)
generated_amount = [ratio * total_amount for ratio in generated_recipe[1]]
for item,quantity_ratio in zip(generated_recipe[0],generated_recipe[1]):
    recipe[item] = quantity_ratio * total_amount
test_abv= cocktail_embedding_maker.calculate_recipe_abv(generated_recipe[0], generated_recipe[1])
taste_info_result = cocktail_embedding_maker.get_taste_info(recipe)
taste_info_result.pop('ID')
print(f"taste_info_result: {json.dumps(taste_info_result,indent=4)}")
print(f"user_preference: {json.dumps(user_preference,indent=4)}")
print(f"test_abv: {test_abv}")

In [ ]:

generated_recipe = recipe_generation_model.generate_recipe(seed_ingredient, user_preference, max_length=5)
total_amount = 200
generated_amount = [ratio * total_amount for ratio in generated_recipe[1]]
result = []
recipe = {}
for item,quantity_ratio in zip(generated_recipe[0],generated_recipe[1]):
    recipe[item] = quantity_ratio * total_amount
    result.append(f"{item} ={quantity_ratio * total_amount}")
print(f"Generated ingredient list: {generated_recipe[0]}")
print(f"Generated Recipe : {result}")
test_abv= cocktail_embedding_maker.calculate_recipe_abv(generated_recipe[0], generated_recipe[1])
taste_info_result = cocktail_embedding_maker.get_taste_info(recipe)
taste_info_result.pop('ID')
print(f"taste_info_result: {json.dumps(taste_info_result,indent=4)}")
print(f"user_preference: {json.dumps(user_preference,indent=4)}")
print(f"test_abv: {test_abv}")